In [61]:
import math

from ax.service.utils.instantiation import ObjectiveProperties
from ax.service.ax_client import AxClient
from ax.models.torch.botorch_modular.surrogate import Surrogate
from ax.modelbridge.registry import Models
from ax.modelbridge.generation_strategy import GenerationStep, GenerationStrategy
from gpytorch.constraints.constraints import GreaterThan
from gpytorch.likelihoods.gaussian_likelihood import (
    GaussianLikelihood,
)
from gpytorch.priors import GammaPrior, LogNormalPrior
from gpytorch.kernels import RBFKernel, MaternKernel
import numpy as np
import os
from os.path import dirname, abspath, join
import sys
import json
import torch
from time import time
from botorch.acquisition import (
    qNoisyExpectedImprovement,
)
from botorch.acquisition.logei import qLogNoisyExpectedImprovement
from botorch.test_functions import (
    Hartmann,
    Levy,
)
from botorch.models import SingleTaskGP
from utils.synthetic import Embedded

In [6]:
ACQUISITION_FUNCTIONS = {
    'NEI': qNoisyExpectedImprovement,
    'qLogNEI': qLogNoisyExpectedImprovement,
}

INITS = {
    'sobol': Models.SOBOL,
}

TEST_FUNCTIONS = {
    'levy4_1000': (Embedded, dict(function=Levy(dim=4), noise_std=0.0, negate=True, dim=1000))
}

In [18]:
num_init = math.ceil(3 * 1000 ** 0.5)
num_bo = 1000 - num_init

In [57]:
# acquistion function
acq_func = ACQUISITION_FUNCTIONS['qLogNEI']

bound_list = []

for _ in range(1000):
    bound_list.append([0.0, 1.0])
bounds = torch.tensor(bound_list).transpose(1, 0)
acq_kwargs = {"acq_kwargs" : True}

gp_params = {
    "const" : {
        "loc" : 0.0,
        "scale" : 1.0
    },
    "ls" : {
        "loc" : 1.4,
        "scale" : 1.73205
    },
    "noise" : {
        "loc" : -4,
        "scale" : 1.0
    }
}

In [58]:
ls_params = gp_params.get("ls", {})
ops_params = gp_params.get("ops", {})
noise_params = gp_params.get("noise", {})

In [59]:
model_kwargs = {
    'covar_module_class' : RBFKernel,
    'covar_module_options' : dict(
        ard_num_dims = 1000,
        lengthscale_prior = LogNormalPrior(**ls_params),
        lengthscale_constraint = GreaterThan(1e-4),
    ),
    'likelihood_class' : GaussianLikelihood,
    'likelihood_options' : dict(
        noise_prior = LogNormalPrior(**noise_params),
        noise_constraint = GreaterThan(1e-4)
    )
}

In [60]:
init_type = Models.SOBOL
init_kwargs = {
    "seed" : 42
}
steps = [
    GenerationStep(
        model = init_type,
        num_trials = num_init,
        model_kwargs = init_kwargs
    )
]

In [63]:
# optimization setuo
model = SingleTaskGP
bo_step = GenerationStep(
    model=Models.BOTORCH_MODULAR,
    num_trials=num_bo,
    model_kwargs={
        "surrogate" : Surrogate(
            botorch_model_class=model,
            covar_module_class=model_kwargs['covar_module_class'],
            covar_module_options=model_kwargs['covar_module_options'],
            likelihood_class=model_kwargs['likelihood_class'],
            likelihood_options=model_kwargs['likelihood_options']
        ),
        "botorch_acqf_class" : acq_func,
        "acquisition_options" : {"prune_baseline": True}
    }
)
steps.append(bo_step)